###**Q1**.  Use [Lucidchart](https://sjsu.edu/it/services/applications/lucidchart.php) to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [1]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
# Your code here . . .
import sqlite3
import pandas as pd
from pathlib import Path

Path('store.db').touch()
conn = sqlite3.connect('store.db')
conn.execute("PRAGMA foreign_keys = 1")
c = conn.cursor()

In [15]:
# Create Product Table
SQL_CreateProductTable = '''
    CREATE TABLE IF NOT EXISTS product (
        product_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        price REAL
    );
'''

# Create Customer Table
SQL_CreateCustomerTable = '''
    CREATE TABLE IF NOT EXISTS customer (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL
    );
'''

# Create Order_Item Table
SQL_CreateOrderItemTable = '''
    CREATE TABLE IF NOT EXISTS order_item (
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER,
        FOREIGN KEY (order_id) REFERENCES purchase_order(order_id),
        FOREIGN KEY (product_id) REFERENCES product(product_id)
    );
'''

# Create Purchase_Order Table
SQL_CreatePurchaseOrderTable = '''
    CREATE TABLE IF NOT EXISTS purchase_order (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        date TEXT NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
    );
'''

c.execute(SQL_CreateProductTable)
c.execute(SQL_CreateCustomerTable)
c.execute(SQL_CreateOrderItemTable)
c.execute(SQL_CreatePurchaseOrderTable)

In [16]:
# 2.3 Load the data in the csv files into the corresponding table.
# Your code here . . .
product_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
customer_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
order_item_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')
purchase_order_data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')

# Insert data into tables
product_data.to_sql('product', conn, if_exists='replace', index=False)
customer_data.to_sql('customer', conn, if_exists='replace', index=False)
order_item_data.to_sql('order_item', conn, if_exists='replace', index=False)
purchase_order_data.to_sql('purchase_order', conn, if_exists='replace', index=False)

3

In [17]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
# Your code here . . .
conn.commit()

In [18]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
# Your code here . . .
c.execute("SELECT * FROM product")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM customer")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM order_item")
results = c.fetchall()
print(results)

c.execute("SELECT * FROM purchase_order")
results = c.fetchall()
print(results)

[(0, 'bicycle', 400), (1, 'helmet', 45), (2, 'gloves', 23), (3, 'chain', 48)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [19]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
# Your code here . . .
query = '''
    SELECT customer.name AS customer_name,
           purchase_order.date AS purchase_date,
           product.name AS product_name,
           order_item.quantity AS quantity
    FROM customer
    JOIN purchase_order ON customer.customer_id = purchase_order.customer_id
    JOIN order_item ON purchase_order.order_id = order_item.order_id
    JOIN product ON order_item.product_id = product.product_id
    WHERE customer.name = 'Wendy Lee'
'''
c.execute(query)
results = c.fetchall()
results

[('Wendy Lee', '2020-10-19', 'bicycle', 1),
 ('Wendy Lee', '2020-10-19', 'helmet', 1),
 ('Wendy Lee', '2020-10-20', 'gloves', 2),
 ('Wendy Lee', '2020-10-20', 'chain', 1)]

In [20]:
# Additional tasks
# Appending a new row to customer table
new_customer = '''
    INSERT OR IGNORE INTO customer (customer_id, name, email)
    VALUES (4, 'Thi Pham', 'thip@abc.com')
'''
c.execute(new_customer)
conn.commit()

c.execute("SELECT * FROM customer")
results = c.fetchall()
print(results)

# Deleting row(s) with "XXX"

c.execute("DELETE FROM customer WHERE customer_id = 2")
conn.commit()

c.execute("SELECT * FROM customer")
results = c.fetchall()
print(results)

[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu'), (4, 'Thi Pham', 'thip@abc.com')]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (3, 'Godric Gryffindor', 'gg@hogwards.edu'), (4, 'Thi Pham', 'thip@abc.com')]
